<a href="https://colab.research.google.com/github/KIM-Jaegyeong/SBA2019_SeoulIT/blob/master/19_10_02%EA%B3%A0%EC%88%98%EC%A4%80API%EC%82%AC%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)
import os, warnings
# 이미지 처리 분야에서 가장 유명한 신경망 모델인 CNN 을 이용
import tensorflow as tf
import time

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [2]:
print(mnist.train.images.shape)
print(mnist.train.labels.shape)
print(mnist.test.images.shape)
print(mnist.test.labels.shape)
print(mnist.validation.images.shape)
print(mnist.validation.labels.shape)

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)
(5000, 784)
(5000, 10)


## 01.신경망 모델 구성

In [0]:
# 기존 모델에서는 입력 값을 28x28 하나의 차원으로 구성하였으나,
# CNN 모델을 사용하기 위해 2차원 평면과 특성치의 형태를 갖는 구조로 만듭니다.
# None는 입력데이터의 개수, 마지막 차원 1은 채널(색특징)의 개수.
# MNIST는 회색조의 이미지로 색상이 한개
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool) # dropout를 수행할지 안할지 결정

## CNN 계층 구성
- W1: 가중치
 - W1 [3 3 1 32]
   - [3,3] : 커널크기
   - 1 : 입력값 채널수
   - 32 : 필터의 개수 (커널의 개수)
- L1 : 계층
 - (?, 28, 28, 32)
   -? : 데이터 개수는 확정되지 않음.
   - 28, 28 : 커널을 통해 만들어진 이미지 픽셀
   - 32 : 커널(필터)의 개수
- Pool : 플링 계층
   - (?, 14, 14, 32)
   - ? : 데이터 개수가 정해져 있지 않음.
   - 14, 14 : 커널을 통해 만들어진 이미지 픽셀
   - 32 : 커널(필터)의 개수


In [0]:
# 원래 코드
# W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
# L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
# L1 = tf.nn.relu(L1) # 활성화 함수
# L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


In [4]:
L1 = tf.layers.conv2d(X,32,[3,3])
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2])
L1 = tf.layers.dropout(L1, 0.7, is_training)


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.dropout instead.


In [0]:
# W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
# L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
# L2 = tf.nn.relu(L2)
# L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.layers.conv2d(L1, 64, [3,3])
L2 = tf.layers.max_pooling2d(L2, [2,2], [2,2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

In [6]:
# W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
# L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
# L3 = tf.matmul(L3, W3)
# L3 = tf.nn.relu(L3)
# L3 = tf.nn.dropout(L3, keep_prob)
L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [0]:
# 최종 출력값 L3 에서의 출력 256개를 입력값으로 받아서 0~9 레이블인 10개의 출력값을 만듭니다.
# W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
# model = tf.matmul(L3, W4)
model = tf.layers.dense(L3, 10, activation=None)


In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [9]:
%%time
#########
# 신경망 모델 학습
######
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
for epoch in range(10):
    total_cost = 0
    for i in range(total_batch):
       batch_xs, batch_ys = mnist.train.next_batch(batch_size)
 # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
       batch_xs = batch_xs.reshape(-1, 28, 28, 1)
       _, cost_val = sess.run([optimizer, cost],feed_dict={X: batch_xs, Y: batch_ys,is_training: True})
       total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1),
 'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print('최적화 완료!')


Epoch: 0001 Avg. cost = 0.172
Epoch: 0002 Avg. cost = 0.049
Epoch: 0003 Avg. cost = 0.031
Epoch: 0004 Avg. cost = 0.021
Epoch: 0005 Avg. cost = 0.015
Epoch: 0006 Avg. cost = 0.015
Epoch: 0007 Avg. cost = 0.012
Epoch: 0008 Avg. cost = 0.008
Epoch: 0009 Avg. cost = 0.008
Epoch: 0010 Avg. cost = 0.007
최적화 완료!
CPU times: user 10min 51s, sys: 11.8 s, total: 11min 3s
Wall time: 5min 43s


In [10]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
 feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
 Y: mnist.test.labels,
 is_training: False}))

정확도: 0.989
